# Load previously trained model

In [2]:
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + (1 - dice_loss(y_true, y_pred))

def unet_down_one_block(inputs, num_filters):
    x = Conv2D(num_filters, (3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def unet_max_pool(inputs):
    x = MaxPooling2D((2, 2), strides=(2, 2))(inputs)
    return x

def unet_up_one_block(up_input, down_input, num_filters):
    x = UpSampling2D((2,2))(up_input)
    x = concatenate([down_input, x], axis=3)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, (3,3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

def get_unet(input_shape = (256, 256, 3),
             num_classes = 1,
             initial_filters = 32,
             central_filters = 1024):
    
    num_filters = initial_filters
    
    out_list    = [Input(shape=input_shape)]
    down_interim_list = []
    
    while num_filters <= central_filters/2:
        x = unet_down_one_block(out_list[-1], num_filters)
        down_interim_list.append(x)
        num_filters = num_filters * 2
        y = unet_max_pool(x)
        out_list.append(y)
    
    x = unet_down_one_block(out_list[-1], num_filters)
    out_list.append(x)
    num_filters = int(num_filters / 2)
    
    while num_filters >= initial_filters:
        x = unet_up_one_block(out_list[-1], down_interim_list.pop(), num_filters)
        out_list.append(x)
        num_filters = int(num_filters / 2)
    
    classify = Conv2D(num_classes, (1,1), activation = 'sigmoid')(out_list[-1])
    
    model = Model(inputs=out_list[0], outputs=classify)
    
    model.compile(optimizer=RMSprop(lr=0.0001),
                  loss=bce_dice_loss,
                  metrics=[dice_loss])
    
    return model

model = get_unet(input_shape=(512,512,3), initial_filters=16)
model.load_weights(filepath='weights/best_weights_512.hdf5')

NameError: name 'Input' is not defined

# Generate predictions

In [ ]:
import pickle

In [ ]:
with open("data/all_imgs_resized_512.pkl", "rb") as f:
    all_imgs = pickle.load(f)

In [ ]:
preds = model.predict(np.array(all_imgs, np.float32)/255, batch_size=16)
preds = np.squeeze(preds, axis=3)

# Worst 10

In [ ]:
orig_width = 1918
orig_height= 1280

def dice_loss_numpy(y_true, y_pred):
    return 2*np.sum(np.multiply(y_true, y_pred))/(np.sum(y_pred) + np.sum(y_true))

In [ ]:
dice_loss_vals=[]
for i in range(len(ids_list)):
    tmp = np.copy(preds[i,:,:])
    tmp = cv2.resize(tmp, (orig_width, orig_height))
    tmp = (tmp > 0.5).astype(int)
    mask = cv2.imread('data/train_masks/{}_mask.png'.format(ids_list[i]), cv2.IMREAD_GRAYSCALE)/255
    dice_loss_vals.append(dice_loss_numpy(mask, tmp))

In [ ]:
sorted_dice_loss_vals = list(sorted(zip(ids_list, dice_loss_vals, range(len(ids_list))), key=lambda x:x[1]))

In [ ]:
def plt_ith_diff(i):
    mask = cv2.imread('data/train_masks/{}_mask.png'.format(sorted_dice_loss_vals[i][0]), cv2.IMREAD_GRAYSCALE)/255
    tmp = np.copy(preds[sorted_dice_loss_vals[i][2],:,:])
    tmp = cv2.resize(tmp, (orig_width, orig_height))
    tmp = (tmp > 0.5).astype(int)
    
    print(dice_loss_numpy(mask, tmp))
    
    plt.figure(figsize=(15, 15))
    plt.imshow(np.subtract(tmp, mask));

In [ ]:
plt_ith_diff(0)